## Por que usar algoritmos de otimização?
- Os algoritmos de otimização são usados quando meu universo de soluções para um determinado problema é muito grande,
tornando impraticável usar uma força bruta para tentar encontrar a melhor
- No caso do exemplo abaixo, é possível ter cerca de 300 milhões de resultados diferentes

### Qual o problema?
Um grupo de amigos que mora em diferentes estados irão para um mesmo evento na cidade de guarulhos.
Como forma de diminuir os custos, eles pretendem se reunir no aeroporto de guarulhos para pegar um mesmo transporte e dividir os custos, tanto no ida como na volta do evento.
Portanto eles precisam descobrir qual o melhor horário de voo que cada um deve tomar, saindo dos seus estados de origem, para chegar no aeroporto de guarulhos sem ter que esperar muito tempo lá, até que todos cheguem para que eles possam ir no evento

### Objetivo
- Criar uma agenda ÓTIMA de voos que tem como origem diferentes estados e destino à guarulhos e vice e versa
- Nesse caso eu não tenho uma solução definitiva ou resultado claro, portanto eu preciso executar meu algoritmo até encontrar a melhor solução dentro daquelas que foram encontradas
- É provável que não seja literalmente A MELHOR, porém é suficientemente boa
- O contrário disso seria por exemplo se eu QUISESSE que todos gastassem até R$200 e esperassem no máximo 1 hora no aeroporto

### Representação
Definição das variáveis iniciais que irão ser usadas para o problema e da variável que deverá ser otimizada 
(agenda)

In [37]:
import time
import random
import math

pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

voos = {}

### Criação do dict de voos
A partir do arquivo **voos.txt** iremos fazer um for para ler as suas linhas e pegar os horários e preço de
cada um dos voos

In [38]:
for linha in open('voos.txt'):
    _origem, _destino, _saida, _chegada, _preco = linha.split(',')
    voos.setdefault((_origem, _destino), [])
    voos[(_origem, _destino)].append((_saida, _chegada, int(_preco)))

### Função de imprimir agenda
Função final, que vai basicamente imprimir o resultado final do voo que cada pessoa vai tomar, baseado na agenda que ela recebeu, o desafio está justamente em otimizar essa agenda

In [39]:
def get_ida_volta(i, id_voo, solucao):
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][solucao[id_voo]]
    id_voo += 1
    volta = voos[(destino, origem)][solucao[id_voo]]
    
    return ida, volta

In [40]:
def imprimir_agenda(agenda):
    id_voo = -1
    for i in range(len(agenda) // 2):
        nome = pessoas[i][0] 
        origem = pessoas[i][1]
        ida, volta = get_ida_volta(i, id_voo, agenda)
        print('%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s' % (nome, origem, ida[0], ida[1], ida[2],
                                                       volta[0], volta[1], volta[2]))
        id_voo += 2

In [41]:
agenda = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3] ##Exemplo aleatório de agenda / solução
imprimir_agenda(agenda) 

    Amanda       CWB  8:04-10:11 R$ 95 12:08-14:05 R$142
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 11:08-14:38 R$262
   Jessica       CNF  9:42-11:32 R$169 12:08-14:47 R$231
     Paulo       GYN 13:37-15:08 R$250 11:07-13:24 R$171


In [42]:
def get_minutos(hora): #transforma as horas em minutos
    x = time.strptime(hora, '%H:%M')
    minutos = x[3] * 60 + x[4]
    return minutos

## Função de custo
- É basicamente o coração dos algoritmos de otimização
- Eu transformo todas as minhas variáveis (dimensões) em um único valor, que vai expressar matematicamente o quão bom ou quão ruim é a solução encontrada

## Função de custo para os voos
- Nesse caso as minhas principais variáveis serão o **PREÇO** e **TEMPO**, ambas devem serem expressadas em um 
único valor que deverá ser minimizado (quanto mais barato e menos tiver que esperar melhor)
- Cada problema é muito particular e tem sua própria função de custo
- Nesse caso, nós assumimos que as pessoas só vão deixar o aeroporto para ir ao evento quando TODOS chegarem e só iram deixar o evento e ir ao aeroporto quando O PRIMEIRO tiver que pegar o voo

In [43]:
def aplicar_penalidade(ultima_chegada, primeira_partida):
    penalidade = 0
    if ultima_chegada > primeira_partida:
        penalidade += 50
        
    return penalidade

In [95]:
def funcao_custo(solucao):
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = 1439
    custo = 0
    
    id_voo = -1
    for i in range(len(solucao) // 2):
        ida, volta = get_ida_volta(i, id_voo, solucao)
        
        preco_total += ida[2]
        preco_total += volta[2]
        
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])
            
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
        
        id_voo += 2
            
    total_espera = 0
    id_voo = -1
    for i in range(len(solucao) // 2):
        ida, volta = get_ida_volta(i, id_voo, solucao)
        
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida
        
        id_voo += 2
        
    custo = preco_total + total_espera   
    custo += aplicar_penalidade(ultima_chegada, primeira_partida)
    
    return custo

### Penalidade
- Podemos aplicar penalidades baseado na nossa regra de negócio, por exemplo, na função de custo em questão, 
caso o grupo de amigos tenha que dormir no local (passe para o dia seguinte) a gente adicionar +50 como penalidade.

In [96]:
funcao_custo(agenda) #meu objetivo será minimizar esse valor (cada minuto representa 1 real)

4635

### Pesquisa Randômica
A pesquisa randômica é a busca por soluções de forma aleatória, geralmente é usada como uma baseline para avaliar os algoritmos de otimização

### Definição do domínio do problema
- O domínio será o universo de possibilidade que o meu problema terá, qual o RANGE do domínio do meu problema em questão.
- No caso dos voos, meu domínio é de (0,9). Sendo assim, é possível escolher qualquer número INTEIRO entre 0 e 9 e esse número será um índice no meu array de voos

In [97]:
def pesquisa_randomica(dominio, funcao_custo):
    melhor_custo = 999999999 #meu objetivo é minimizar

    for i in range(0, 10000): #quantidade de soluções que serão geradas

        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        custo = funcao_custo(solucao)
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao

    return melhor_solucao

In [98]:
dominio = [(0,9)] * (len(pessoas) * 2)
solucao_randomica = pesquisa_randomica(dominio, funcao_custo)
custo_randomica = funcao_custo(solucao_randomica)
imprimir_agenda(solucao_randomica)

print('menor custo', custo_randomica)

    Amanda       CWB 11:16-13:29 R$ 83  8:23-10:28 R$149
     Pedro       GIG 10:30-14:57 R$290  7:57-11:15 R$347
    Marcos       POA 13:40-15:38 R$137  8:19-11:16 R$122
  Priscila       FLN 12:05-15:30 R$330 15:23-18:49 R$150
   Jessica       CNF  9:42-11:32 R$169  9:11-10:42 R$172
     Paulo       GYN 12:18-14:56 R$172  9:31-11:43 R$210
menor custo 3509


In [ ]:
def plotar_busca(resultados):
    plot.show()